In [1]:
import pandas as pd
import numpy as np
import pickle

# Load df_train_2

In [2]:
face_result_df_train_2 = pd.read_csv('train_set_2_face_result.csv', usecols=['id', 'face_detect'], dtype={'id': str}).rename(columns={'face_detect': "face_prediction"})
text_result_df_train_2 = pd.read_csv('train_set_2_text_result.csv', usecols=['id', 'text_label_1', 'label'], dtype={'id': str}).rename(columns={'text_label_1': "text_prediction_score"})
image_result_df_train_2 = pd.read_csv('train_set_2_image_result.csv', usecols=['id', '1'], dtype={'id': str}).rename(columns={'1': "image_prediction_score"})

In [3]:
image_result_df_train_2.head()

,id,image_prediction_score
0,7053444957239069978,0.985132
1,7053444957239069978,0.740242
2,7053444957239069978,0.859779
3,7053444957239069978,0.244381
4,7053444957239069978,0.995782


Group by 'id':

In [4]:
agg_face_result = face_result_df_train_2.groupby('id').sum().reset_index()
agg_face_result['face_prediction'] = agg_face_result['face_prediction'].map(lambda x: 1 if x > 0 else 0)

agg_image_result = image_result_df_train_2.groupby('id').mean().reset_index()

Make prediction:

In [5]:
threshold = 0.5
agg_image_result['image_prediction'] = agg_image_result['image_prediction_score'].map(lambda x: 1 if x >= threshold else 0)
text_result_df_train_2['text_prediction'] = text_result_df_train_2['text_prediction_score'].map(lambda x: 1 if x >= threshold else 0)

In [6]:
text_result_df_train_2.head()

,id,label,text_prediction_score,text_prediction
0,6746395734930640129,0,0.047635,0
1,7071073830848023834,1,0.784508,1
2,6954907710366387458,1,0.750353,1
3,7066615161271881006,0,0.064409,0
4,6925795138379713794,0,0.140355,0


In [7]:
# agg_image_result.head()

In [8]:
combine_df_train = pd.merge(left=text_result_df_train_2, 
                    right= pd.merge(left=agg_image_result, 
                                    right=agg_face_result, 
                                    how='inner', 
                                    on='id'), 
                    how='inner', on='id')
combine_df_train['label'] = combine_df_train['label'].astype(int)

In [9]:
combine_df_train

,id,label,text_prediction_score,text_prediction,image_prediction_score,image_prediction,face_prediction
0,6746395734930640129,0,0.047635,0,0.314742,0,0
1,7071073830848023834,1,0.784508,1,0.996597,1,0
2,6954907710366387458,1,0.750353,1,0.941859,1,1
3,7066615161271881006,0,0.064409,0,0.013347,0,0
4,6925795138379713794,0,0.140355,0,0.548625,1,0
...,...,...,...,...,...,...,...
3108,7058856419461795073,1,0.863431,1,0.739146,1,0
3109,6974710162594548993,0,0.065865,0,0.015326,0,0
3110,6992891507560713498,0,0.155982,0,0.120283,0,0
3111,7106113028952755482,1,0.975066,1,0.924086,1,1


# Load df_test

In [10]:
face_result_df_test = pd.read_csv('test_set_face_result.csv', usecols=['id', 'face_detect'], dtype={'id': str}).rename(columns={'face_detect': "face_prediction"})
text_result_df_test = pd.read_csv('test_set_text_result.csv', usecols=['id', 'text_label_1', 'label'], dtype={'id': str}).rename(columns={'text_label_1': "text_prediction_score"})
image_result_df_test = pd.read_csv('test_set_image_result.csv', usecols=['id', '1'], dtype={'id': str}).rename(columns={'1': "image_prediction_score"})

Group by 'id':

In [11]:
agg_face_result_test = face_result_df_test.groupby('id').sum().reset_index()
agg_face_result_test['face_prediction'] = agg_face_result_test['face_prediction'].map(lambda x: 1 if x > 0 else 0)

agg_image_result_test = image_result_df_test.groupby('id').mean().reset_index() # can using max()

Make prediction:

In [12]:
threshold = 0.5
agg_image_result_test['image_prediction'] = agg_image_result_test['image_prediction_score'].map(lambda x: 1 if x >= threshold else 0)
text_result_df_test['text_prediction'] = text_result_df_test['text_prediction_score'].map(lambda x: 1 if x >= threshold else 0)

In [13]:
combine_df_test = pd.merge(left=text_result_df_test, 
                    right= pd.merge(left=agg_image_result_test, 
                                    right=agg_face_result_test, 
                                    how='inner', 
                                    on='id'), 
                    how='inner', on='id')
combine_df_test['label'] = combine_df_test['label'].astype(int)

In [14]:
combine_df_test

,id,label,text_prediction_score,text_prediction,image_prediction_score,image_prediction,face_prediction
0,7070502982655216923,1,0.837612,1,0.326651,0,0
1,7005431713895566618,0,0.143845,0,0.000477,0,0
2,7048453986843675906,1,0.678705,1,0.180475,0,0
3,6863250722155138309,0,0.185896,0,0.472381,0,0
4,7042150256489925890,0,0.124335,0,0.030126,0,0
...,...,...,...,...,...,...,...
1330,7055620670448815387,1,0.780432,1,0.768180,1,0
1331,7050668314317589761,0,0.053739,0,0.661314,1,0
1332,7107300625355967771,1,0.864828,1,0.978852,1,1
1333,6950137272515808514,0,0.060117,0,0.004670,0,0


In [15]:
3113/1335

2.331835205992509

In [16]:
35/15

2.3333333333333335

# Evaluation

In [17]:
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score

In [18]:
# df_test = combine_df_train
df_test = combine_df_test

### Text

In [19]:
accuracy_score(df_test['label'], df_test['text_prediction'])

0.9715355805243445

In [20]:
f1_score(df_test['label'], df_test['text_prediction'])

0.9641509433962264

In [21]:
roc_auc_score(df_test['label'], df_test['text_prediction_score'])

0.9980778867658386

### Face

In [22]:
accuracy_score(df_test['label'], df_test['face_prediction'])

0.6898876404494382

In [23]:
f1_score(df_test['label'], df_test['face_prediction'])

0.39473684210526316

### Image

In [24]:
accuracy_score(df_test['label'], df_test['image_prediction'])

0.8898876404494382

In [25]:
f1_score(df_test['label'], df_test['image_prediction'])

0.8590604026845637

In [26]:
roc_auc_score(df_test['label'], df_test['image_prediction_score'])

0.9512244916897893

# Fusion model

#### Logistic Regression

In [27]:
from sklearn.linear_model import LogisticRegression

In [28]:
clf = LogisticRegression(random_state=2022)
clf.fit(combine_df_train[['text_prediction_score','image_prediction_score', 'face_prediction']], combine_df_train['label'])

LogisticRegression(random_state=2022)

In [29]:
y_test_pred = clf.predict(df_test[['text_prediction_score','image_prediction_score', 'face_prediction']])

In [35]:
write_pickle(clf, 'model/fusion_clf_lgbm_lr')

### LightGBM

In [74]:
import lightgbm as lgb

In [75]:
params = {
    'objective': 'binary',
    'metric': 'binary_logloss',
    'verbose': -1,
    'boosting': 'gbdt',
    'learning_rate': 0.1,
    'num_leaves': 8,
    'min_data_in_leaf': 20,
    'bagging_fraction': 0.8,
    'feature_fraction': 0.8,
    'lambda_l1': 0.0001,
    'lambda_l2': 0.0001,
    'max_bin': 20,
    'num_boost_round': 100,
    'n_jobs': 8
}

In [76]:
model = lgb.LGBMClassifier(**params)
model.fit(combine_df_train[['text_prediction_score','image_prediction_score', 'face_prediction']], combine_df_train['label'])

[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] lambda_l1 is set=0.0001, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.0001
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] lambda_l2 is set=0.0001, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.0001
[LightGBM] [Warning] num_iterations is set=100, num_boost_round=100 will be ignored. Current value: num_iterations=100


D:\Program Files\anaconda3\envs\python_env\lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


LGBMClassifier(bagging_fraction=0.8, boosting='gbdt', feature_fraction=0.8,
               lambda_l1=0.0001, lambda_l2=0.0001, max_bin=20,
               metric='binary_logloss', min_data_in_leaf=20, n_jobs=8,
               num_boost_round=100, num_leaves=8, objective='binary',
               verbose=-1)

In [77]:
y_test_pred = model.predict(df_test[['text_prediction_score','image_prediction_score', 'face_prediction']])

#### Eval

In [30]:
f1_score(df_test['label'], y_test_pred)

0.9778188539741219

In [31]:
accuracy_score(df_test['label'], y_test_pred)

0.9820224719101124

In [32]:
model.predict_proba(np.array([0,0.5,1]).reshape(1, -1))[:,1][0]

NameError: name 'model' is not defined

# LightGBM feature important

In [104]:
model

LGBMClassifier(bagging_fraction=0.8, boosting='gbdt', feature_fraction=0.8,
               lambda_l1=0.0001, lambda_l2=0.0001, max_bin=20,
               metric='binary_logloss', min_data_in_leaf=20, n_jobs=8,
               num_boost_round=100, num_leaves=8, objective='binary',
               verbose=-1)

In [99]:
feature_name = model.feature_name_
feature_gain = model.feature_importances_

In [101]:
feature_name

['text_prediction_score', 'image_prediction_score', 'face_prediction']

In [102]:
feature_gain

array([ 79, 526,  95])

In [34]:
def write_pickle(object, file_name):
    with open(file_name, "wb") as handle:
        pickle.dump(object, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [106]:
write_pickle(model, 'model/fusion_clf_lgbm')